In [1]:
import os
from dotenv import load_dotenv
from datetime import datetime

import openai
from IPython.display import HTML, display
from ipywidgets import widgets
from langchain.chains import ConversationalRetrievalChain
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import DirectoryLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

load_dotenv()

True

In [2]:
loader = DirectoryLoader("/home/jinzy/work/automation/ask_gpt/testdocs", glob="**/*.pdf", show_progress=True, use_multithreading=True)

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 1000,
    chunk_overlap  = 100,
    length_function = len,
)
txt_docs = loader.load_and_split(text_splitter=text_splitter)

embeddings = OpenAIEmbeddings()
txt_docsearch = Chroma.from_documents(txt_docs, embeddings)

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.5, max_tokens=2048)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:04<00:00,  1.60s/it]


In [4]:
from langchain.chains.summarize import load_summarize_chain

# 创建总结链
#vchain = load_summarize_chain(llm, chain_type="refine", verbose=True)

In [6]:
from langchain.chains import LLMChain
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate
from langchain.prompts.chat import (
  ChatPromptTemplate,
  SystemMessagePromptTemplate,
  HumanMessagePromptTemplate
)
from langchain.memory import ConversationBufferMemory


system_template = """
Use the following context to answer the user's question.
If you don't know the answer, say you don't, don't try to make it up. 请用中文回答.
-----------
{context}
-----------
{chat_history}
"""

# 构建初始 messages 列表，这里可以理解为是 openai 传入的 messages 参数
messages = [
  SystemMessagePromptTemplate.from_template(system_template),
  HumanMessagePromptTemplate.from_template('{question}')
]

# 初始化 prompt 对象
prompt = ChatPromptTemplate.from_messages(messages)

# 初始化问答链
question_generator = LLMChain(llm=llm, prompt=prompt)
doc_chain = load_qa_chain(llm, chain_type="refine")

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
# chain = ConversationalRetrievalChain(
#     retriever=txt_docsearch.as_retriever(),
#     question_generator=question_generator,
#     combine_docs_chain=doc_chain,
#     memory=memory
# )
chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=txt_docsearch.as_retriever(), 
                                              chain_type='refine', memory=memory)

In [7]:
query = "请用中文解释一下RPM产品中的任务挖掘功能"
result = chain({"question": query})
print(result)

{'question': '请用中文解释一下RPM产品中的任务挖掘功能', 'chat_history': [HumanMessage(content='请用中文解释一下RPM产品中的任务挖掘功能', additional_kwargs={}, example=False), AIMessage(content='iS-RPM的任务挖掘功能通过分析桌面用户交互数据和从IT系统后台数据中挖掘的流程详细信息，打通业务断点，提高流程数据的完整可见性。企业领导者能够轻松实现海量流程数据的可视化分析，发现重复、低效的任务在哪些地方减慢了流程的运行，并在正确的位置分配资源以解决这些问题，为端到端流程优化做出数据驱动的决策。iS-RPM的任务挖掘功能可以帮助决策者确定在流程的什么阶段实施RPA机器人，更有效地拓展RPA项目，提升流程自动化收益。', additional_kwargs={}, example=False)], 'answer': 'iS-RPM的任务挖掘功能通过分析桌面用户交互数据和从IT系统后台数据中挖掘的流程详细信息，打通业务断点，提高流程数据的完整可见性。企业领导者能够轻松实现海量流程数据的可视化分析，发现重复、低效的任务在哪些地方减慢了流程的运行，并在正确的位置分配资源以解决这些问题，为端到端流程优化做出数据驱动的决策。iS-RPM的任务挖掘功能可以帮助决策者确定在流程的什么阶段实施RPA机器人，更有效地拓展RPA项目，提升流程自动化收益。'}


In [9]:
query = "请给一个RPM产品的实际案例介绍"
#result = chain({"question": query, "chat_history": chat_history})
result = chain({"question": query})
print(result)
#print(result['answer'])

{'question': '请给一个RPM产品的实际案例介绍', 'chat_history': [HumanMessage(content='请用中文解释一下RPM产品中的任务挖掘功能', additional_kwargs={}, example=False), AIMessage(content='iS-RPM的任务挖掘功能通过分析桌面用户交互数据和从IT系统后台数据中挖掘的流程详细信息，打通业务断点，提高流程数据的完整可见性。企业领导者能够轻松实现海量流程数据的可视化分析，发现重复、低效的任务在哪些地方减慢了流程的运行，并在正确的位置分配资源以解决这些问题，为端到端流程优化做出数据驱动的决策。iS-RPM的任务挖掘功能可以帮助决策者确定在流程的什么阶段实施RPA机器人，更有效地拓展RPA项目，提升流程自动化收益。', additional_kwargs={}, example=False), HumanMessage(content='请给一个RPM产品的实际案例介绍', additional_kwargs={}, example=False), AIMessage(content='iS-RPM\'s task mining feature is a part of the product\'s overall process mining capabilities, which helps businesses discover automation opportunities and optimize their end-to-end processes. The feature uses a "Discovery Robot" to capture screen shots and log data of user actions and applies machine learning to identify tasks with high automation potential. This helps decision-makers determine at which stage of the process to implement RPA robots, effectively expanding RPA projects and imp

In [10]:
print(result['answer'])

iS-RPM's task mining feature is a part of the product's overall process mining capabilities, which helps businesses discover automation opportunities and optimize their end-to-end processes. The feature uses a "Discovery Robot" to capture screen shots and log data of user actions and applies machine learning to identify tasks with high automation potential. This helps decision-makers determine at which stage of the process to implement RPA robots, effectively expanding RPA projects and improving process automation benefits.

In addition, iS-RPM's process mining capabilities, including the task mining feature, provide businesses with a comprehensive view of their processes, enabling them to identify automation opportunities, optimize processes, and improve their overall efficiency. The product captures events' system logs or event logs that contain other related information about each step, such as a supplier's detailed information for invoices or the priority of payment requests, to in

In [12]:
query = "请介绍一个RPM产品的实际案例"
#result = chain({"question": query, "chat_history": chat_history})
result = chain({"question": '请用中文回答: ' + query})

print(result['answer'])

An example of iS-RPM in action is a Chinese manufacturing company that used the software to integrate and optimize their business processes. The company had dispersed business processes across multiple departments and systems, leading to difficulties in process management and low efficiency. iS-RPM helped the company identify best practices for business processes and implement corresponding improvement measures. The task mining feature also helped decision-makers determine when to implement RPA robots in the process, effectively extending RPA projects to improve process automation benefits. Overall, iS-RPM helped the Chinese manufacturing company optimize their business processes, increase efficiency, reduce costs, and improve transparency. With its ability to provide real-time data collection, visual process maps, and task mining capabilities, iS-RPM allowed for data-driven decision-making, making it easier for business leaders to identify and prioritize automation opportunities for e

In [13]:
result

{'question': '请用中文回答: 请介绍一个RPM产品的实际案例',
 'chat_history': [HumanMessage(content='请用中文解释一下RPM产品中的任务挖掘功能', additional_kwargs={}, example=False),
  AIMessage(content='iS-RPM的任务挖掘功能通过分析桌面用户交互数据和从IT系统后台数据中挖掘的流程详细信息，打通业务断点，提高流程数据的完整可见性。企业领导者能够轻松实现海量流程数据的可视化分析，发现重复、低效的任务在哪些地方减慢了流程的运行，并在正确的位置分配资源以解决这些问题，为端到端流程优化做出数据驱动的决策。iS-RPM的任务挖掘功能可以帮助决策者确定在流程的什么阶段实施RPA机器人，更有效地拓展RPA项目，提升流程自动化收益。', additional_kwargs={}, example=False),
  HumanMessage(content='请给一个RPM产品的实际案例介绍', additional_kwargs={}, example=False),
  AIMessage(content='iS-RPM\'s task mining feature is a part of the product\'s overall process mining capabilities, which helps businesses discover automation opportunities and optimize their end-to-end processes. The feature uses a "Discovery Robot" to capture screen shots and log data of user actions and applies machine learning to identify tasks with high automation potential. This helps decision-makers determine at which stage of the process to implement RPA robots, effectively expanding RPA pr

In [14]:
query = "请介绍一个RPM产品的实际案例"
#result = chain({"question": query, "chat_history": chat_history})
result = chain({"question": query + ' 请用中文回答。'})

print(result['answer'])

Yes, I can provide a practical case of using iS-RPM, which is an RPM product from Yisaqi. The task mining feature of iS-RPM can help decision-makers identify which stage of the process to implement RPA robots, enabling more effective expansion of RPA projects and improving process automation benefits. The "Discovery Robot" installed on the user's desktop allows enterprises to capture any operation by employees on their desktops and analyze the captured event information, providing bottom-level technical support for discovering and mining automated business processes. iS-RPM breaks through the "manual" demand management bottleneck in traditional automation business management activities by automatically identifying high-frequency tasks and giving evaluators a global perspective to prioritize the development of automated processes, continuously outputting automation opportunities from the bottom up. The product also provides ROI analysis, time analysis, and other analysis panels to help 